# Data Enrichment

Data enrichment and data aggregation are the processes involved around joining merging datasets, creating new columns, calculating values on certain windows, grouping into bins, or even changing the values. All of these processes will assist in the data analysis by providing specific insight into the data. Things such as severity of anomalous data, rolling averages, cumulative sums, or quantities grouped by ages can all make the data easier to interpret. We'll focus on the following for this class:
- Querying and Merging dataframes
- Aggregating Dataframes


In [1]:
import pandas as pd

In [2]:
weather = pd.read_csv('https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas/master/ch_04/data/nyc_weather_2018.csv')

In [3]:
snow_data = weather.query('datatype == "SNOW" and value > 0')
snow_data.head()

,attributes,datatype,date,station,value
124,",,N,",SNOW,2018-01-01T00:00:00,GHCND:US1NYWC0019,25.0
723,",,N,",SNOW,2018-01-04T00:00:00,GHCND:US1NJBG0015,229.0
726,",,N,",SNOW,2018-01-04T00:00:00,GHCND:US1NJBG0017,10.0
730,",,N,",SNOW,2018-01-04T00:00:00,GHCND:US1NJBG0018,46.0
737,",,N,",SNOW,2018-01-04T00:00:00,GHCND:US1NJES0018,10.0


As a reminder, in previous chapters we have used the following syntax to generate the same output. 

In [4]:
snow_data_mask = weather[
    (weather.datatype == 'SNOW') & (weather.value > 0)
]
snow_data_mask.head()

,attributes,datatype,date,station,value
124,",,N,",SNOW,2018-01-01T00:00:00,GHCND:US1NYWC0019,25.0
723,",,N,",SNOW,2018-01-04T00:00:00,GHCND:US1NJBG0015,229.0
726,",,N,",SNOW,2018-01-04T00:00:00,GHCND:US1NJBG0017,10.0
730,",,N,",SNOW,2018-01-04T00:00:00,GHCND:US1NJBG0018,46.0
737,",,N,",SNOW,2018-01-04T00:00:00,GHCND:US1NJES0018,10.0


In [5]:
snow_data_mask.equals(snow_data)

True

The method that you choose largely depends on preference. 

## Merging DataFrames

Another crutial part of data analysis is combining datasets together, creating a more complete understanding of the data. There are two types of merges that we typically talk about. Using the venacular common in from SQL: 
- `join`: a join is where you two sets of columns through one or values in the respective rows matching on some condition. 
- `union`: a merge where the columns are the same on two datasets, and bundled together on the same table.

Join is the more complex between the two, so we'll mostly talk about that.

### Joining Datasets
So far in this class, we have only worked with a single dataset. Joins provide us the ability to take two seperate tables or dataframes with related information, and combine them into a single table. For the weather data we've been using, we might perform a join to attach a physical location to the weather measurements using the weather station's id to gain a bettr idea of how the weather patterns are destributed.

The most common types of joins are described below using venn diagrams.  

![joins diagram](Assets/joins.jpg "Joins Diagram")

Think of the circles as the complete set of rows for each dataframe and the shaded region as the rows that are returned as a result of the join. Described breifly, 
- Inner joins return only the rows that are present in both dataframes, 
- Left (Right) joins return all of the rows from the left (right) dataframe, leaving all the values as null or missing (depending on your language) for the columns from the right (left) table if there was no matching ID found in that table
- Full or Outer joins return all rows from both tables, leaving missing values in the columns on both sides for missing IDs in either table (Think the left and right joins happening at the same time)

Since it can be easier to understand something by doing, let's at least look at some examples:

In [6]:
# reading in the weather station data
stations = pd.read_csv("https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas/master/ch_04/data/weather_stations.csv")

In [7]:
stations.head()

,id,name,latitude,longitude,elevation
0,GHCND:US1CTFR0022,"STAMFORD 2.6 SSW, CT US",41.0641,-73.5770,36.6
1,GHCND:US1CTFR0039,"STAMFORD 4.2 S, CT US",41.0378,-73.5682,6.4
2,GHCND:US1NJBG0001,"BERGENFIELD 0.3 SW, NJ US",40.9213,-74.0020,20.1
3,GHCND:US1NJBG0002,"SADDLE BROOK TWP 0.6 E, NJ US",40.9027,-74.0834,16.8
4,GHCND:US1NJBG0003,"TENAFLY 1.3 W, NJ US",40.9147,-73.9775,21.6


The first thing we need to do to join two dataframes is determine the rows that we can join on. Note the row called ID. This looks nearly identical to the type of values we had in some of the weather data. 

In [8]:
weather.head()

,attributes,datatype,date,station,value
0,",,N,",PRCP,2018-01-01T00:00:00,GHCND:US1CTFR0039,0.0
1,",,N,",PRCP,2018-01-01T00:00:00,GHCND:US1NJBG0015,0.0
2,",,N,",SNOW,2018-01-01T00:00:00,GHCND:US1NJBG0015,0.0
3,",,N,",PRCP,2018-01-01T00:00:00,GHCND:US1NJBG0017,0.0
4,",,N,",SNOW,2018-01-01T00:00:00,GHCND:US1NJBG0017,0.0


Here, that columns is called station in our weather data. The next thing we should do is look at the shape to determine how to perform the join. 

In [9]:
print(f"Weather: {weather.shape}")
print(f"Stations: {stations.shape}")

Weather: (80256, 5)
Stations: (262, 5)


As a habbit and personal preference, I will typically take the take the table with less rows and join it to the table with more rows, but before we do that, it could also be beneficial to look at the number of unique values just to make sure that we don't lose any important information. 

In [10]:
print(f"Unique stations from Weather: {weather.station.unique().shape}")
print(f"Unique stations from Stations: {stations.id.unique().shape}")

Unique stations from Weather: (109,)
Unique stations from Stations: (262,)


We could probably go a step further and double check that all of station IDs in `weather` are also present in `stations`, but I'm not that concerned about about that right now. We can move on to actually executing the join. There are several methods that you can use to perform the different joins, but I will showcase `merge` since it is capable of performing all the joins we are interested in for this class.

In [11]:
joined_tables = weather.merge(stations, left_on='station', right_on='id')

In [12]:
joined_tables.head()

,attributes,datatype,date,station,value,id,name,latitude,longitude,elevation
0,",,N,",PRCP,2018-01-01T00:00:00,GHCND:US1CTFR0039,0.0,GHCND:US1CTFR0039,"STAMFORD 4.2 S, CT US",41.0378,-73.5682,6.4
1,",,N,",PRCP,2018-01-02T00:00:00,GHCND:US1CTFR0039,0.0,GHCND:US1CTFR0039,"STAMFORD 4.2 S, CT US",41.0378,-73.5682,6.4
2,",,N,",PRCP,2018-01-03T00:00:00,GHCND:US1CTFR0039,0.0,GHCND:US1CTFR0039,"STAMFORD 4.2 S, CT US",41.0378,-73.5682,6.4
3,",,N,",DAPR,2018-01-05T00:00:00,GHCND:US1CTFR0039,2.0,GHCND:US1CTFR0039,"STAMFORD 4.2 S, CT US",41.0378,-73.5682,6.4
4,",,N,",MDPR,2018-01-05T00:00:00,GHCND:US1CTFR0039,15.5,GHCND:US1CTFR0039,"STAMFORD 4.2 S, CT US",41.0378,-73.5682,6.4


Note that now we have all of the weather data from before (datatype, date, value, etc.) on the left side of the table, and all of the location data on the right (latitude, longitude, and elevation). Unfortunitely we do have two columns with identical informaiton, but this could be solved easiy by dropping one of the columns after the fact or by renaming the one column beforehand. If the joining columns have the same name, python only includes one column with that name. 

In [24]:
weather.merge(stations.rename({'id':'station'}, axis='columns').drop('elevation', axis='columns'), on='station').head()

,attributes,datatype,date,station,value,name,latitude,longitude
0,",,N,",PRCP,2018-01-01,GHCND:US1CTFR0039,0.0,"STAMFORD 4.2 S, CT US",41.0378,-73.5682
1,",,N,",PRCP,2018-01-02,GHCND:US1CTFR0039,0.0,"STAMFORD 4.2 S, CT US",41.0378,-73.5682
2,",,N,",PRCP,2018-01-03,GHCND:US1CTFR0039,0.0,"STAMFORD 4.2 S, CT US",41.0378,-73.5682
3,",,N,",DAPR,2018-01-05,GHCND:US1CTFR0039,2.0,"STAMFORD 4.2 S, CT US",41.0378,-73.5682
4,",,N,",MDPR,2018-01-05,GHCND:US1CTFR0039,15.5,"STAMFORD 4.2 S, CT US",41.0378,-73.5682


In [25]:
stations.rename({'id':'station'}, axis='columns').drop('elevation', axis='columns')

,station,name,latitude,longitude
0,GHCND:US1CTFR0022,"STAMFORD 2.6 SSW, CT US",41.06410,-73.57700
1,GHCND:US1CTFR0039,"STAMFORD 4.2 S, CT US",41.03780,-73.56820
2,GHCND:US1NJBG0001,"BERGENFIELD 0.3 SW, NJ US",40.92130,-74.00200
3,GHCND:US1NJBG0002,"SADDLE BROOK TWP 0.6 E, NJ US",40.90270,-74.08340
4,GHCND:US1NJBG0003,"TENAFLY 1.3 W, NJ US",40.91470,-73.97750
...,...,...,...,...
257,GHCND:USW00054787,"FARMINGDALE REPUBLIC AIRPORT, NY US",40.73417,-73.41694
258,GHCND:USW00094728,"NY CITY CENTRAL PARK, NY US",40.77898,-73.96925
259,GHCND:USW00094741,"TETERBORO AIRPORT, NJ US",40.85000,-74.06139
260,GHCND:USW00094745,"WESTCHESTER CO AIRPORT, NY US",41.06694,-73.70750


Note that with this inner join, there are no missing values on either side of the table (at least none that isn't just bad or missing data)

In [17]:
joined_tables.query('id.isna() or station.isna()').shape

(0, 10)

We can probably for that to happen by performing a left or right join. 

In [20]:
right_joined_tables = weather.merge(stations, left_on='station', right_on='id', how='right')
right_joined_tables.query("id.isna() or station.isna()")

,attributes,datatype,date,station,value,id,name,latitude,longitude,elevation
0,NaN,NaN,NaN,NaN,NaN,GHCND:US1CTFR0022,"STAMFORD 2.6 SSW, CT US",41.06410,-73.577000,36.6
344,NaN,NaN,NaN,NaN,NaN,GHCND:US1NJBG0001,"BERGENFIELD 0.3 SW, NJ US",40.92130,-74.002000,20.1
345,NaN,NaN,NaN,NaN,NaN,GHCND:US1NJBG0002,"SADDLE BROOK TWP 0.6 E, NJ US",40.90270,-74.083400,16.8
718,NaN,NaN,NaN,NaN,NaN,GHCND:US1NJBG0005,"WESTWOOD 0.8 ESE, NJ US",40.98300,-74.015900,15.8
719,NaN,NaN,NaN,NaN,NaN,GHCND:US1NJBG0006,"RAMSEY 0.6 E, NJ US",41.05860,-74.134100,112.2
...,...,...,...,...,...,...,...,...,...,...
50877,NaN,NaN,NaN,NaN,NaN,GHCND:USC00309400,"WHITE PLAINS MAPLE M, NY US",41.01667,-73.733330,45.7
50878,NaN,NaN,NaN,NaN,NaN,GHCND:USC00309466,WILLETS POINT,40.80000,-73.766667,16.8
50879,NaN,NaN,NaN,NaN,NaN,GHCND:USC00309576,"WOODLANDS ARDSLEY, NY US",41.01667,-73.850000,42.7
50880,NaN,NaN,NaN,NaN,NaN,GHCND:USW00014708,"HEMPSTEAD MITCHELL FIELD AFB, NY US",40.73333,-73.600000,38.1


Notice now that we have a set of rows for which the weather station data is present, but under the weather data, we have a bunch of missing values. This indicates to us that the some of the weather stations present in the `stations` dataframe are *not* present in the `weather` dataframe. Doing a quick bit of math, we can confirm that the missing rows matches up with what we expected. Performing an outer join is essentially the same as a left and a right join together, with the above behavior expected for both the left and right tables. 

## Window Functions

Window functions are a very interesting and useful function that can provide some insight. Sometimes, we want to know the maximum or the average of an entire column, but it can be just as interesting to know the rolling average of some sort of data. Window functions allow us to perform calculations  on a group of rows that are close to each otherin some way. Using the weather data, we can calculat the average rainfall over the past week. 

In [66]:
snowfall = weather.query("datatype == 'PRCP' and station == 'GHCND:USW00094728'")\
    .set_index("date")\
    .assign(rolling_average=lambda x: x.value.rolling(7).mean())

In [67]:
snowfall.query('rolling_average > 0')

,attributes,datatype,station,value,rolling_average
date,,,,,
2018-01-07T00:00:00,",,W,2400",PRCP,GHCND:USW00094728,0.0,2.757143
2018-01-08T00:00:00,",,W,2400",PRCP,GHCND:USW00094728,0.3,2.800000
2018-01-09T00:00:00,",,W,2400",PRCP,GHCND:USW00094728,0.0,2.800000
2018-01-10T00:00:00,",,W,2400",PRCP,GHCND:USW00094728,0.0,2.800000
2018-01-11T00:00:00,",,W,2400",PRCP,GHCND:USW00094728,0.0,0.042857
...,...,...,...,...,...
2018-12-27T00:00:00,",,W,2400",PRCP,GHCND:USW00094728,0.0,6.342857
2018-12-28T00:00:00,",,W,2400",PRCP,GHCND:USW00094728,29.2,4.671429
2018-12-29T00:00:00,",,W,2400",PRCP,GHCND:USW00094728,0.0,4.600000


## Aggregation
Another important function of data analysis is the idea of aggregation. This referes to the process of taking the data and rolling it up into a single value or set of values instead of looking at the individual measurements. We might want the summed, average, or maximum value of the dataset. We can use fairly simple function calls that we have acctually used to a degree before. I will use pivot tables to combine the different aggregations together in our weather data example. 

In [21]:
weather.date = pd.to_datetime(weather.date)
weather_pivot = weather.set_index(['date', 'station'])\
    .pivot(columns='datatype', values='value')[['PRCP', 'SNOW', 'TAVG', 'TMAX', 'TMIN']]
weather_pivot.query('station == "GHCND:USW00094728"')[['PRCP', 'SNOW']].sum()


datatype
PRCP    1665.3
SNOW    1007.0
dtype: float64

The above gives us the totals over the entire dataset. We could make this perform the same logic to calculate things like the average per day, or find the days with the most or the least. The method calls are fairly intuitive, and can be quickly found in Panda's documentation. To make this slightly more intersting, we can use a method called `groupby` to group the data by some patitioning feature and perform the aggregation function on each group respectively. 

In [22]:
weather_pivot.groupby("station")[['PRCP', 'SNOW']].sum()

datatype,PRCP,SNOW
station,,
GHCND:US1CTFR0039,1424.8,543.0
GHCND:US1NJBG0003,1590.7,1046.0
GHCND:US1NJBG0010,1233.2,0.0
GHCND:US1NJBG0015,1539.0,770.0
GHCND:US1NJBG0017,1447.8,1019.0
...,...,...
GHCND:USW00054787,1291.5,0.0
GHCND:USW00094728,1665.3,1007.0
GHCND:USW00094741,1611.3,1192.0


or we could group by the date

In [113]:
weather_pivot.groupby('date').min()

datatype,PRCP,SNOW,TAVG,TMAX,TMIN
date,,,,,
2018-01-01,0.0,0.0,-11.3,-10.6,-22.2
2018-01-02,0.0,0.0,-7.8,-10.0,-20.0
2018-01-03,0.0,0.0,-6.8,-10.6,-16.1
2018-01-04,0.0,0.0,-4.0,-3.8,-15.6
2018-01-05,0.0,0.0,-9.2,-8.8,-17.2
...,...,...,...,...,...
2018-12-27,0.0,0.0,4.2,4.4,-2.8
2018-12-28,0.0,0.0,8.0,6.1,-2.8
2018-12-29,0.0,0.0,10.2,12.2,1.1


Additionally, you can group by multiple columns. It will create a set of nested groups that can be aggregated together for further separation of the metrics being calculated. 

There is also a pandas class called `Grouper` that seems to allow for even more complex grouping functionality 

# playground

In [50]:
df = pd.DataFrame([range(10), [None for i in range(10)]]).T.rename({0: "index", 1: "empty"}, axis='columns')

In [58]:
df.value_counts('index')

index
0.0    1
1.0    1
2.0    1
3.0    1
4.0    1
5.0    1
6.0    1
7.0    1
8.0    1
9.0    1
dtype: int64

In [43]:
df.isna()

,index,empty
0,False,True
1,False,True
2,False,True
3,False,True
4,False,True
5,False,True
6,False,True
7,False,True
8,False,True
9,False,True


In [48]:
for col in df.isna().columns:
    if df[col].all(): 
        df = df.drop(col, axis='columns')

In [66]:
df.filename = ['file.csv' for _ in range(10)]

In [67]:
df

,index,empty
0,0.0,NaN
1,1.0,NaN
2,2.0,NaN
3,3.0,NaN
4,4.0,NaN
5,5.0,NaN
6,6.0,NaN
7,7.0,NaN
8,8.0,NaN
9,9.0,NaN


| | good | bad | okay | 
| - | - | - | - | 
file1.csv| 3 | 5 | 6 |
file2.csv| 3 | 5 | 6 |
file3.csv| 3 | 5 | 6 |

| cat | count |
| - | - |
| good| 3 | 
| bad | 5 | 
| okay | 34 | 

In [ ]:
csv_lines = [] 
with open('file.cs') as file: 
    line = file.readline()
    line = line.split(',')
    if len(line) == 5: 
        csv_lines.append(line)

df = pd.DataFrame(csv_lines[1:], columns=csv_lines[0])